In [1]:
import keras
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
import pandas as pd
batch_size = 32

Using TensorFlow backend.


# Data Pull
Data comes from ![Kaggle](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data)
My file structure is as follows: 
```
data/
    test/
        unk/
            12500 cat/dog unkown pictures
    train/
        cat/
            11250 cat pictures
        dog/
            11250 dog pictures
    valid/
        cat/
            1250 cat pictures
        dog/
            1250 dog pictures
sample/
    data/
        train/
            cat/
                1138 cat pictures
            dog/
                1138 dog pictures
        valid/
            cat/
                112 cat pictures
            dog/
                112 dog pictures
```

In [2]:
#path = "sample/data/"
path = "data/"

In [15]:
idg = image.ImageDataGenerator(rotation_range=0.1,horizontal_flip=True, channel_shift_range=0.2)
idg_test = image.ImageDataGenerator()
trn_batch = idg.flow_from_directory(path+"train/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
val_batch = idg.flow_from_directory(path+"valid/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
tst_batch = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size, shuffle=False)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [14]:
??tst_batch

In [4]:
#Use VGG16 from Keras
vgg16 = VGG16()
#Finetune to look at just two classes instead of 1000
vgg16.layers.pop()
for layer in vgg16.layers:
    layer.trainable = False 
m = Dense(1, activation="sigmoid")(vgg16.layers[-1].output)
vgg16 = Model(vgg16.input, m)
#Just guessing on the loss here
vgg16.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [5]:
vgg16.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)

Epoch 1/5
704/703 [==============================] - 124s - loss: 0.1041 - acc: 0.9624 - val_loss: 0.0985 - val_acc: 0.9696
Epoch 2/5
704/703 [==============================] - 117s - loss: 0.0766 - acc: 0.9734 - val_loss: 0.1019 - val_acc: 0.9680
Epoch 3/5
704/703 [==============================] - 116s - loss: 0.0617 - acc: 0.9772 - val_loss: 0.0957 - val_acc: 0.9676
Epoch 4/5
704/703 [==============================] - 119s - loss: 0.0540 - acc: 0.9802 - val_loss: 0.1106 - val_acc: 0.9672
Epoch 5/5
704/703 [==============================] - 118s - loss: 0.0474 - acc: 0.9836 - val_loss: 0.0937 - val_acc: 0.9704


In [16]:
predictions = vgg16.predict_generator(tst_batch, steps=(tst_batch.n/batch_size))

In [17]:
submission = pd.DataFrame(columns=['id','label'])

In [18]:
predictions[0][0]

0.52462566

In [19]:
#trimming the extremes so nothing is predicting too strongly a 1 or a 0
for i in range(0,len(predictions)):
    pred_id = tst_batch.filenames[i].split("/")[1].split(".")[0]
    pred_label = predictions[i][0]
    if pred_label > 0.97: 
        pred_label = 0.97
    if pred_label < 0.03:
        pred_label = 0.03
    submission = submission.append({"id":pred_id, "label":pred_label}, ignore_index=True)

In [20]:
submission.to_csv("Submission_Dog_Cat_Redux_17_10_19_02.csv", index=False)

In [21]:
tst_batch.filenames[0]

'unk/8606.jpg'

In [22]:
predictions[0]

array([ 0.52462566], dtype=float32)